In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3

from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4

from KNN.UserKNNCBFRecommender import UserKNNCBFRecommender


#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender

In [2]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')



#Load playlistsIDS
train_df= pd.read_csv('all/train.csv',low_memory = False)
all_userID=train_df.playlist_id.unique()

In [8]:
import implicit

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=200, regularization=0.1,iterations=5)

# train the model on a sparse matrix of item/user/confidence weights
item_user_data=URM_train.T
print("ITEM USER ={}".format(item_user_data.shape))
model.fit(item_user_data)

# recommend items for a user
user_items = item_user_data.T.tocsr()
print("USER ITEM ={}".format(user_items.shape))


#uno per volta predico tutti gli utenti e estraggo dalla lista delle 10 predizioni, dalla tupla, la track
rec_matrix=np.zeros((50446,10))

for u in all_userID:
    rec = model.recommend(u, user_items)
    for x in range (10):
        rec_matrix[u,x]=rec[x][0]
print(rec_matrix)   

ITEM USER =(20635, 50446)


100%|██████████| 5.0/5 [00:04<00:00,  1.34it/s]


USER ITEM =(50446, 20635)
[[15187. 13250.  7545. ... 20152. 12768.  4202.]
 [ 2336.  4202.  7077. ...  5534.  1422.  8195.]
 [ 3855.  4916.  5817. ...  7611.  3880. 18490.]
 ...
 [15542.   229.  7465. ...  6491.  2627. 18059.]
 [11638.  4106.  7153. ... 17780. 19991. 14591.]
 [ 7889.  2627.   648. ...  5097. 16494.  8866.]]


In [4]:
import numpy as np
import scipy.sparse as sps



def precision(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score



def recall(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score



def MAP(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

In [5]:
def evaluate_algorithm(URM_test, recMatrix, at=10):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0

    num_eval = 0

    URM_test = sps.csr_matrix(URM_test)

    n_users = URM_test.shape[0]


    for user_id in range(n_users):

        if user_id % 10000 == 0:
            print("Evaluated user {} of {}".format(user_id, n_users))

        start_pos = URM_test.indptr[user_id]
        end_pos = URM_test.indptr[user_id+1]

        if end_pos-start_pos>0:

            relevant_items = URM_test.indices[start_pos:end_pos]

            recommended_items = recMatrix[user_id]
            num_eval+=1

            is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

            cumulative_precision += precision(is_relevant, relevant_items)
            cumulative_recall += recall(is_relevant, relevant_items)
            cumulative_MAP += MAP(is_relevant, relevant_items)


    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval

    print("Recommender performance is: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP))

    result_dict = {
        "precision": cumulative_precision,
        "recall": cumulative_recall,
        "MAP": cumulative_MAP,
    }

    return result_dict

In [6]:
evaluate_algorithm(URM_test,rec_matrix)

Evaluated user 0 of 50446
Evaluated user 10000 of 50446
Evaluated user 20000 of 50446
Evaluated user 30000 of 50446
Evaluated user 40000 of 50446
Evaluated user 50000 of 50446
Recommender performance is: Precision = 0.0105, Recall = 0.0205, MAP = 0.0037


{'MAP': 0.003655664736114788,
 'precision': 0.01053121124431573,
 'recall': 0.020542532529209664}

In [7]:
import implicit

factors=[10,50,100,200,500]
reg=[0.01, 0.001,0.1,0.0001]
it=[5,20,40]
item_user_data=URM_train.T
user_items = item_user_data.T.tocsr()





for f in factors:
    for r in reg:
        for i in it:
            print("fator={}, regularization={}, iterations={}".format(f,r,i))
            # initialize a model
            model = implicit.als.AlternatingLeastSquares(factors=f, regularization=r,iterations=i)

            # train the model on a sparse matrix of item/user/confidence weights
            #print("ITEM USER ={}".format(item_user_data.shape))
            model.fit(item_user_data)

            # recommend items for a user
            #print("USER ITEM ={}".format(user_items.shape))


            #uno per volta predico tutti gli utenti e estraggo dalla lista delle 10 predizioni, dalla tupla, la track
            rec_matrix=np.zeros((50446,10))

            for u in all_userID:
                rec = model.recommend(u, user_items)
                for x in range (10):
                    rec_matrix[u,x]=rec[x][0]
            #print(rec_matrix)   
            evaluate_algorithm(URM_test,rec_matrix)

  0%|          | 0/5 [00:00<?, ?it/s]

fator=10, regularization=0.01, iterations=5


100%|██████████| 5.0/5 [00:01<00:00,  3.19it/s]


Evaluated user 0 of 50446
Evaluated user 10000 of 50446
Evaluated user 20000 of 50446
Evaluated user 30000 of 50446
Evaluated user 40000 of 50446
Evaluated user 50000 of 50446
Recommender performance is: Precision = 0.0301, Recall = 0.0600, MAP = 0.0238
fator=10, regularization=0.01, iterations=20


100%|██████████| 20.0/20 [00:05<00:00,  3.80it/s]


...

Evaluated user 0 of 50446
Evaluated user 10000 of 50446
Evaluated user 20000 of 50446
Evaluated user 30000 of 50446
Evaluated user 40000 of 50446
Evaluated user 50000 of 50446
Recommender performance is: Precision = 0.0003, Recall = 0.0006, MAP = 0.0001
fator=500, regularization=0.0001, iterations=20


100%|██████████| 20.0/20 [00:55<00:00,  2.66s/it]


Evaluated user 0 of 50446
Evaluated user 10000 of 50446
Evaluated user 20000 of 50446
Evaluated user 30000 of 50446
Evaluated user 40000 of 50446
Evaluated user 50000 of 50446
Recommender performance is: Precision = 0.0004, Recall = 0.0006, MAP = 0.0001
fator=500, regularization=0.0001, iterations=40


100%|██████████| 40.0/40 [01:53<00:00,  2.77s/it]


Evaluated user 0 of 50446
Evaluated user 10000 of 50446
Evaluated user 20000 of 50446
Evaluated user 30000 of 50446
Evaluated user 40000 of 50446
Evaluated user 50000 of 50446
Recommender performance is: Precision = 0.0002, Recall = 0.0003, MAP = 0.0002
